In [2]:
import duckdb
import pandas as pd
import os
import sys
from datetime import datetime

In [31]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [43]:
arquivo = 'z0019_2.csv'
data_ingestao = datetime.now()
df = pd.read_csv(f'../landing/{arquivo}', sep=";")
df['nome_arquivo'] = arquivo
df['data_ingestao'] = data_ingestao

df.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10003,PREGO,BT10,100,60,z0019_2.csv,2025-09-23 21:24:03.660288
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-09-23 21:24:03.660288
2,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-09-23 21:24:03.660288


In [44]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bronze_produtos(
            NATBR VARCHAR,
            MAKTX VARCHAR,
            WERKS VARCHAR,
            MAINS VARCHAR,
            LABST VARCHAR,
            nome_arquivo VARCHAR,
            data_ingestao TIMESTAMP
            )
            """)

In [45]:
con.execute("INSERT INTO bronze_produtos SELECT * FROM df")

In [49]:
resultado = con.execute("select * from bronze_z0019").fetchdf()
resultado.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-09-23 21:23:49.511440
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-09-23 21:23:49.511440
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-09-23 21:23:49.511440
3,10003,PREGO,BT10,100,60,z0019_2.csv,2025-09-23 21:24:03.660288
4,10004,SERRA,BT50,100,200,z0019_2.csv,2025-09-23 21:24:03.660288
5,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-09-23 21:24:03.660288


In [47]:
con.execute("alter table bronze_produtos rename to bronze_z0019")

In [50]:
con.close()